In [ ]:
import numpy as np
import random
import cv2
import os
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K
from sklearn.metrics import classification_report
#from fl_mnist_implementation_tutorial_utils import *

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data(path="mnist.npz")
# resize and normalize
x_train = np.reshape(x_train, [-1, 784])
x_train = x_train.astype('float32') / 255
x_test = np.reshape(x_test, [-1, 784])
x_test = x_test.astype('float32') / 255
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
import torchvision.datasets as Datasets
from torch.utils.data import DataLoader
client2_train_data = Datasets.SVHN('/data',split='train',download=True)
client2_x_train,client2_y_train=client2_train_data.data,[client2_train_data[i][1] for i in range(len(client2_train_data))]
client2_test_data = Datasets.SVHN('/data',split='test',download=True)
client2_x_test,client2_y_test=client2_test_data.data,[client2_test_data[i][1] for i in range(len(client2_test_data))]

client2_x_train,client2_x_train = np.reshape(client2_x_train,[-1,3*32*32]),client2_x_train.astype('float32') / 255
client2_x_test,client2_x_test = np.reshape(client2_x_test,[-1,3*32*32]),client2_x_test.astype('float32') / 255
client2_y_train,client2_y_test = to_categorical(client2_y_train),to_categorical(client2_y_test)

client1_train_data = Datasets.MNIST('/data',train=True,download=True)
client1_x_train,client1_y_train=client1_train_data.data.numpy(),client1_train_data.targets.numpy()
client1_test_data = Datasets.MNIST('/data',train=False,download=True)
client1_x_test,client1_y_test=client1_test_data.data.numpy(),client1_test_data.targets.numpy()

client1_x_train,client1_x_train = np.reshape(client1_x_train,[-1,28*28]),client1_x_train.astype('float32') / 255
client1_x_test,client1_x_test = np.reshape(client1_x_test,[-1,28*28]),client1_x_test.astype('float32') / 255
client1_y_train,client1_y_test = to_categorical(client1_y_train),to_categorical(client1_y_test)


Using downloaded and verified file: /data/train_32x32.mat
Using downloaded and verified file: /data/test_32x32.mat


In [ ]:

clients_batched = dict()
dataset1 = tf.data.Dataset.from_tensor_slices((list(client1_x_train), list(client1_y_train)))
dataset2 = tf.data.Dataset.from_tensor_slices((list(client2_x_train), list(client2_y_train)))
clients_batched['client_1'] = dataset1.shuffle(len(client1_y_train)).batch(32)
clients_batched['client_2'] = dataset2.shuffle(len(client2_y_train)).batch(32)


In [ ]:
class SimpleMLP:
    @staticmethod
    def build(shape, classes):
        model = Sequential()
        model.add(Dense(200, input_shape=(shape,)))
        model.add(Activation("relu"))
        model.add(Dense(200))
        model.add(Activation("relu"))
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        return model
        
def weight_scalling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
    #get the bs
    bs = list(clients_trn_data[client_name])[0][0].shape[0]
    #first calculate the total training data points across clinets
    global_count = sum([tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy() for client_name in client_names])*bs
    # get the total number of data points held by a client
    local_count = tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy()*bs
    return local_count/global_count

def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final



def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
        
    return avg_grad


def test_model(X_test, Y_test,  model, comm_round):
    cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    #logits = model.predict(X_test, batch_size=100)
    logits = model.predict(X_test)
    loss = cce(Y_test, logits)
    acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1))
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    return acc, loss


In [ ]:
clients_batched['client_2']

<_BatchDataset element_spec=(TensorSpec(shape=(None, 3, 32, 32), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.float32, name=None))>

In [ ]:
lr = 0.01 
comms_round = 10
loss='categorical_crossentropy'
metrics = ['accuracy']
optimizer = SGD(learning_rate=lr, momentum=0.9)  
#initialize global model
smlp_global = SimpleMLP()
global_model = smlp_global.build(784, 10)
global_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 200)               157000    
                                                                 
 activation (Activation)     (None, 200)               0         
                                                                 
 dense_1 (Dense)             (None, 200)               40200     
                                                                 
 activation_1 (Activation)   (None, 200)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                2010      
                                                                 
 activation_2 (Activation)   (None, 10)                0         
                                                                 
Total params: 199,210
Trainable params: 199,210
Non-trai

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data(path="mnist.npz")
x_train1 =[]
for imgs in range(len(x_train)):
  x_train1.append(x_train[imgs].flatten())

x_test1 =[]
for imgs in range(len(x_test)):
  x_test1.append(x_test[imgs].flatten())

In [ ]:
def create_clients(image_list, label_list, num_clients=3 ):
    ''' return: a dictionary with keys clients' names and value as 
                data shards - tuple of images and label lists.
        args: 
            image_list: a list of numpy arrays of training images
            label_list:a list of binarized labels for each image
            num_client: number of fedrated members (clients)
            initials: the clients'name prefix, e.g, clients_1 
            
    '''
    initial='clients'
    #create a list of client names
    client_names = ['{}_{}'.format(initial, i+1) for i in range(num_clients)]

    #randomize the data
    data = list(zip(image_list, label_list))
    random.shuffle(data)

    #shard data and place at each client
    size = len(data)//num_clients
    shards = [data[i:i + size] for i in range(0, size*num_clients, size)]

    #number of clients must equal number of shards
    assert(len(shards) == len(client_names))

    return {client_names[i] : shards[i] for i in range(len(client_names))} 
def batch_data(data_shard, bs=32):
    '''Takes in a clients data shard and create a tfds object off it
    args:
        shard: a data, label constituting a client's data shard
        bs:batch size
    return:
        tfds object'''
    #seperate shard into data and labels lists
    data, label = zip(*data_shard)
    dataset = tf.data.Dataset.from_tensor_slices((list(data), list(label)))
    return dataset.shuffle(len(label)).batch(bs)

class SimpleMLP:
    @staticmethod
    def build(shape, classes):
        model = Sequential()
        model.add(Dense(200, input_shape=(shape,)))
        model.add(Activation("relu"))
        model.add(Dense(200))
        model.add(Activation("relu"))
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        return model

In [ ]:
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


In [ ]:
clients = create_clients(x_train1, y_train, num_clients=3)

clients_batched = dict()
for (client_name, data) in clients.items():
    clients_batched[client_name] = batch_data(data)
    
#process and batch the test set  
test_batched = tf.data.Dataset.from_tensor_slices((x_test1, y_test)).batch(len(y_test))

In [ ]:

def weight_scalling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
    #get the bs
    bs = list(clients_trn_data[client_name])[0][0].shape[0]
    #first calculate the total training data points across clinets
    global_count = sum([tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy() for client_name in client_names])*bs
    # get the total number of data points held by a client
    local_count = tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy()*bs
    return local_count/global_count

def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final



def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
        
    return avg_grad


def test_model(X_test, Y_test,  model, comm_round):
    cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    #logits = model.predict(X_test, batch_size=100)
    logits = model.predict(X_test)
    loss = cce(Y_test, logits)
    acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1))
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    
    return acc, loss


In [ ]:
lr = 0.01 
comms_round = 11
loss='categorical_crossentropy'
metrics = ['accuracy']
optimizer = SGD(learning_rate=lr, momentum=0.9)  
#initialize global model
smlp_global = SimpleMLP()
global_model = smlp_global.build(784, 10)
      
#commence global training loop
for comm_round in range(comms_round):
    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()
    
    #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    client_names= list(clients_batched.keys())
    random.shuffle(client_names)
    
    #loop through each client and create new local model
    for client in client_names:
        smlp_local = SimpleMLP()
        local_model = smlp_local.build(784, 10)
        local_model.compile(loss=loss, 
#                      optimizer=optimizer, 
                        optimizer='adam',
                      metrics=metrics)
        
        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)
        
        #fit local model with client's data
        local_model.fit(clients_batched[client], epochs=1, verbose=1)
        
        #scale the model weights and add to list
        scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)
        
        #clear session to free memory after each communication round
        K.clear_session()
        if comm_round==comms_round-1: #last round
          cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    
          logits = local_model.predict(X_test)
          loss_client = cce(Y_test, logits)
          acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1))
          print('{} | Accuracy : {:.3%} | loss: {}'.format(client, acc, loss_client))
          print(client +' classification report')
          print(classification_report(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1),digits=2))




        
    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights = sum_scaled_weights(scaled_local_weight_list)
    
    #update global model 
    global_model.set_weights(average_weights)
        
    #test global model and print out metrics after each communications round
    for(X_test, Y_test) in test_batched:
        global_acc, global_loss = test_model(X_test, Y_test, global_model, comm_round)
    if comm_round==comms_round-1: #last round
      logits = global_model.predict(X_test)
      print('Global classification report')
      print(classification_report(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1),digits=2))




In [ ]:
lr = 0.01 
loss='categorical_crossentropy'
metrics = ['accuracy']
#initialize global model
smlp_global = SimpleMLP()
baseline = smlp_global.build(784, 10)
baseline.compile(loss=loss, 
                        optimizer='adam',
                      metrics=metrics)
dataset = tf.data.Dataset.from_tensor_slices((list(x_train1), list(y_train)))
all_data = dataset.shuffle(len(y_train)).batch(32)
baseline.fit(all_data, epochs=1, verbose=1)

cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
logits = baseline.predict(X_test)
loss_val = cce(Y_test, logits)
acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1))
print(' Accuracy Baseline : {:.3%} | loss: {}'.format( acc, loss_val))
print(classification_report(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1),digits=2))
